In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH99-MyTest")

In [ ]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings
from tqdm import tqdm
import os

# 현재 디렉토리에 DB 저장 경로 설정
current_dir = os.getcwd()
db_path = os.path.join(current_dir, "my_db.db")

# ChromaDB 클라이언트 초기화 (영구 저장소 설정)
client = chromadb.PersistentClient(
    path=db_path, settings=Settings(allow_reset=True, is_persistent=True)
)

# CSV 파일 읽기
df = pd.read_csv("./data/gong.csv")

# Sentence Transformer 임베딩 함수 설정
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# 컬렉션 생성 (이미 존재하는 경우 가져오기)
try:
    collection = client.create_collection(
        name="my_collection", embedding_function=embedding_function
    )
except ValueError:
    # 이미 존재하는 경우 기존 컬렉션 가져오기
    collection = client.get_collection(
        name="my_collection", embedding_function=embedding_function
    )


# 각 행을 문자열로 변환하는 함수
def row_to_string(row):
    return " ".join([f"{col}: {val}" for col, val in row.items()])


# 배치 크기 설정
BATCH_SIZE = 100

# 전체 데이터 처리를 위한 반복 횟수 계산
num_batches = len(df) // BATCH_SIZE + (1 if len(df) % BATCH_SIZE != 0 else 0)

# 배치 처리로 데이터 저장
for i in tqdm(range(num_batches)):
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, len(df))

    batch_df = df.iloc[start_idx:end_idx]

    documents = []
    ids = []

    for idx, row in batch_df.iterrows():
        # 각 행을 문자열로 변환
        doc_string = row_to_string(row)
        documents.append(doc_string)
        ids.append(str(idx))

    # ChromaDB에 배치 추가
    collection.add(documents=documents, ids=ids)

print(f"총 {len(df)}개의 행이 성공적으로 저장되었습니다.")
print(f"데이터베이스가 다음 경로에 저장되었습니다: {db_path}")

In [ ]:
df.head(5)

In [ ]:
# 첫 번째 document 가져오기
result = collection.get(
    ids=["0"],  # 첫 번째 행의 ID
    include=["documents", "embeddings"],  # documents와 embeddings 모두 포함
)

print("Document 내용:")
print(result["documents"][0])  # 첫 번째 document의 텍스트 내용

print("\nEmbedding 벡터 (처음 10개 값):")
print(result["embeddings"][0][:10])  # 임베딩 벡터의 처음 10개 값만 출력

In [ ]:
from openai import OpenAI
import chromadb
from chromadb.utils import embedding_functions
import os

# ChromaDB 클라이언트 연결
client = chromadb.PersistentClient(path="my_db.db")

# Embedding 함수 설정
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# 컬렉션 가져오기
collection = client.get_collection(
    name="my_collection", embedding_function=embedding_function
)

# 쿼리 실행
query = "예비창업자가 지원할 수 있는 지원사업은"
results = collection.query(query_texts=[query], n_results=3)  # 상위 3개 결과 가져오기

# 검색 결과 출력
print("=== 검색된 문서 ===")
for i, doc in enumerate(results["documents"][0]):
    print(f"\nDocument {i+1}:")
    print(doc)

# OpenAI를 사용한 답변 생성
client = OpenAI()
# OpenAI API 키 입력

# 프롬프트 구성
prompt = f"""다음은 예비창업자가 지원할 수 있는 지원사업에 대한 검색 결과입니다. 
이를 바탕으로 예비창업자가 지원할 수 있는 주요 지원사업들을 간단명료하게 요약해서 설명해주세요:

{results['documents'][0]}
"""

# ChatGPT API 호출
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that explains startup support programs in Korean.",
        },
        {"role": "user", "content": prompt},
    ],
    temperature=0.7,
    max_tokens=1000,
)

print("\n=== AI 답변 ===")
print(response.choices[0].message.content)